In [6]:
## Functions
import pandas as pd
import numpy as np

# Import functions from chaintools.py
from chaintools import dividechain
from chaintools import chainget
from chaintools import plot_points

# Function calculates bearing going from point A to point B (e.g., 120 degrees)
def bearing(A_x, A_y, B_x, B_y):
    X = B_x - A_x
    Y = B_y - A_y
    bearing = np.rad2deg(np.arctan2(X, Y))
    bearing = (bearing + 360) % 360
    return bearing

# Function converts a bearing (e.g., 45) to a direction (e.g., Northeast)
def bearing_to_direction_8(bearing):
    if isinstance(bearing, str):
        direction = 'A FATAL ERROR HAS OCCURED'
        print(direction)
    elif bearing >= 22.5 and bearing < 67.5:
        direction = 'Northeast'
    elif bearing >= 67.5 and bearing < 112.5:
        direction = 'East'
    elif bearing >= 112.5 and bearing < 157.5:
        direction = 'Southeast'
    elif bearing >= 157.5 and bearing < 202.5:
        direction = 'South'
    elif bearing >= 202.5 and bearing < 247.5:
        direction = 'Southwest'
    elif bearing >= 247.5 and bearing < 292.5:
        direction = 'West'
    elif bearing >= 292.5 and bearing < 337.5:
        direction = 'Northwest'
    elif bearing > 360 or bearing < 0:
        direction = 'A FATAL ERROR HAS OCCURED'
        print(direction)
    else:
        direction = 'North'
    return direction

def side_street_get(road, street_1, street_2):
    if road == street_1:
        side_street = street_2
    elif road == street_2:
        side_street = street_1
    else:
        print('A FATAL ERROR HAS OCCURED - Did you name your street_1 and street_2 properly?')
    return side_street

def i_am_the_law(length, side_street, distance_from_intersection,
                 direction_from_intersection, driving_direction, x_int, y_int):

    # Round to 0.5m and make clean (e.g., 20.5 metres or 20 metres not 20.0 metres)
    length = round(length*2)/2
    if length*2 % 2 == 0:
        length = int(length)

    distance_from_intersection = round(distance_from_intersection*2)/2
    if distance_from_intersection*2 % 2 == 0:
        distance_from_intersection = int(distance_from_intersection)

    # "an easterly" not "a easterly"
    n = ''
    if direction_from_intersection == 'East':
        n = 'n'
    
    # Round x and y
    x_int = round(x_int, 3)
    y_int = round(y_int, 3)

    if distance_from_intersection < 3:
        legal = (
        f'{driving_direction} side, commencing at its intersection with {side_street}'
        f'(Grid Coordinates X = {x_int}m, Y = {y_int}m and extending in a{n}'
        f'{direction_from_intersection.lower()}erly direction'
        f'following the {driving_direction.lower()}ern kerbline for {length} metres.'
        )
    else:
        legal = (
        f'{driving_direction} side, commencing {distance_from_intersection} metres'
        f'{direction_from_intersection.lower()} of its intersection with {side_street}'
        f'(Grid Coordinates X = {x_int}m, Y = {y_int}m and extending in a{n}'
        f'{direction_from_intersection.lower()}erly direction'
        f'following the {driving_direction.lower()}ern kerbline for {length} metres.'
        )
    return legal

def restriction_get(ruler_name, precision):
    import pandas as pd

    # Chainget (places intersections and items on ruler)
    ruler = pd.read_csv(ruler_name + '.csv')
    intersections = pd.read_csv('intersections.csv')
    items = pd.read_csv('items.csv')

    items = items[items['ruler']==ruler_name].reset_index()
    intersections = intersections[intersections['ruler']==ruler_name].reset_index()

    chain = dividechain(ruler, precision)
    items = chainget(chain, items)
    intersections = chainget(chain, intersections)

    from operator import index
    import numpy as np

    # Finds the distance from item to closest intersection (on same "direction")
    meta_list = [None] * len(items)
    distance_from_intersection = [None] * len(items)
    intersection_index = [None] * len(items)

    intersections_chain = intersections['Chainage']
    items_chain = items['Chainage']

    for i in range(len(items)):
        list = [999999] * len(intersections)
        for j in range(len(intersections)):
            list[j] = abs(items_chain[i] - intersections_chain[j])
        meta_list[i] = list
        distance_from_intersection[i] = min(list)
        intersection_index[i] = np.argmin(list)

    distances = pd.DataFrame(intersection_index, columns = ['intersection_index'])
    distances['distance_from_intersection'] = distance_from_intersection

    processed = pd.concat([items, distances], axis=1)

    # Both start and end of item were calculated
    # Code below drops duplicates keeping the shortest distance
    processed_2 = processed.sort_values('distance_from_intersection')\
                  .drop_duplicates('fid', keep='first')

    processed_3 = pd.merge(processed_2, intersections,
                           left_on='intersection_index', right_index =True,
                           how='left', suffixes = ('', '_intersection'))

    # Obtains bearing and direction from Intersection to Item
    processed_3['bearing_from_intersection'] = processed_3.apply(lambda x: bearing(
                                               x['x_intersection'], x['y_intersection'],
                                               x['x'], x['y']), axis=1)

    processed_3['direction_from_intersection'] = processed_3['bearing_from_intersection'].apply(bearing_to_direction_8)

    processed_3['side_street'] = processed_3.apply(lambda x: side_street_get(
                                                          x['road'], x['street_1'], x['street_2']), axis=1)

    ruler_to_directions = pd.read_csv(ruler_name + '_directions.csv')
    processed_4 = pd.merge(processed_3, ruler_to_directions, on='road', how='left')

    final = processed_4[['fid', 'old_new', 'restriction', 'length', 'road', 'driving_direction',
                         'fid_intersection', 'distance_from_intersection', 'direction_from_intersection',
                         'side_street', 'x_intersection', 'y_intersection']]

    final['legal'] = final.apply(lambda x: i_am_the_law(
                                        x['length'], x['side_street'], x['distance_from_intersection'],
                                        x['direction_from_intersection'], x['driving_direction'],
                                        x['x_intersection'], x['y_intersection']), axis=1)

    return final

In [7]:
bottomleft = restriction_get('Bottomleft', precision = 0.01)
topright = restriction_get('Topright', precision = 0.01)
combined = pd.concat([bottomleft, topright])
combined.to_csv('final.csv')
combined

,fid,old_new,restriction,length,road,driving_direction,fid_intersection,distance_from_intersection,direction_from_intersection,side_street,x_intersection,y_intersection,legal
0,88,New,No-stopping,133.562142,Ngaio Gorge Road,West,11,0.00,North,Kaiwharawhara Road,1749066.529,5431182.391,"West side, commencing at its intersection with..."
1,6,Extension,No-stopping,546.382236,Kaiwharawhara Road,West,11,0.00,North,Ngaio Gorge Road,1749066.529,5431182.391,"West side, commencing at its intersection with..."
2,77,Extension,No-stopping,25.812451,Crofton Road,North,13,0.38,West,Kenya Street,1748584.450,5431770.779,"North side, commencing at its intersection wit..."
3,76,Old,No-stopping,38.085531,Kenya Street,North,13,0.38,West,Crofton Road,1748584.450,5431770.779,"North side, commencing at its intersection wit..."
4,21,Extension,No-stopping,50.566971,Kaiwharawhara Road,West,1,0.51,Southwest,Hutt Road,1749912.060,5430822.440,"West side, commencing at its intersection with..."
5,3,New,No-stopping,431.808017,Ngaio Gorge Road,West,6,1.01,East,Trelissick Crescent,1748458.814,5431272.233,"West side, commencing at its intersection with..."
6,57,New,Bus stop,15.019999,Ngaio Gorge Road,West,7,13.77,Northwest,Trelissick Crescent,1748444.224,5431277.491,"West side, commencing 14 metresnorthwest of it..."
7,11,New,No-stopping,12.993292,Ngaio Gorge Road,West,7,28.81,Northwest,Trelissick Crescent,1748444.224,5431277.491,"West side, commencing 29 metresnorthwest of it..."
8,71,Old,No-stopping,33.912762,Kenya Street,North,9,33.24,Northeast,Trelissick Crescent,1748383.625,5431327.364,"North side, commencing 33 metresnortheast of i..."
9,86,New,P10,5.946617,Kenya Street,North,13,38.46,Southwest,Crofton Road,1748584.450,5431770.779,"North side, commencing 38.5 metressouthwest of..."
